NEURAL COLLABORATIVE FILTERING

In [3]:
import pandas as pd
import tensorflow as tf 
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dot, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu, sigmoid

In [4]:
# Load data
df = pd.read_csv('datasets/training_set_recommendations.csv')

df

,app_id,hours,user_id
0,975370,0.036362,51580
1,304390,0.011520,2586
2,1085660,0.337073,253880
3,703080,0.027447,259432
4,526870,0.007913,23869
...,...,...,...
15998,260230,0.007012,9417270
15999,329430,0.002003,2465811
16000,633230,0.038065,1893954
16001,341940,0.001002,2465811


In [5]:
# Define the NeuMF Model with Functional API
def build_neumf_model(num_users, num_items, latent_dim_gmf, latent_dim_mlp, mlp_layers, dropout_rate_gmf, dropout_rate_mlp):
    # Inputs
    user_input = Input(shape=(1,), dtype='int32', name='user_input')
    item_input = Input(shape=(1,), dtype='int32', name='item_input')

    # GMF Embeddings
    user_embedding_gmf = Embedding(input_dim=num_users, output_dim=latent_dim_gmf, name='gmf_user_embedding')(user_input)
    item_embedding_gmf = Embedding(input_dim=num_items, output_dim=latent_dim_gmf, name='gmf_item_embedding')(item_input)
    gmf_vector = tf.multiply(user_embedding_gmf, item_embedding_gmf)
    gmf_vector = Flatten()(gmf_vector)
    gmf_vector = Dropout(dropout_rate_gmf)(gmf_vector)

    # MLP Embeddings
    user_embedding_mlp = Embedding(input_dim=num_users, output_dim=latent_dim_mlp, name='mlp_user_embedding')(user_input)
    item_embedding_mlp = Embedding(input_dim=num_items, output_dim=latent_dim_mlp, name='mlp_item_embedding')(item_input)
    mlp_vector = Concatenate()([user_embedding_mlp, item_embedding_mlp])
    mlp_vector = Flatten()(mlp_vector)

    # MLP Layers
    for in_size, out_size in zip(mlp_layers[:-1], mlp_layers[1:]):
        mlp_vector = Dense(out_size, activation='relu')(mlp_vector)
    mlp_vector = Dropout(dropout_rate_mlp)(mlp_vector)

    # Combine GMF and MLP vectors
    combined_vector = Concatenate()([gmf_vector, mlp_vector])
    logits = Dense(1, activation='sigmoid')(combined_vector)

    # Define the model
    model = Model(inputs=[user_input, item_input], outputs=logits)
    return model

In [6]:
# Parameters
df['user_id'], user_id_mapping = pd.factorize(df['user_id'])
df['app_id'], app_id_mapping = pd.factorize(df['app_id'])
n_users = df['user_id'].nunique()   # 2000
n_items = df['app_id'].nunique()    # 3723
embedding_dim_gmf = 10
embedding_dim_mlp = 20

# Parameters for MLP layers
mlp_layers = [embedding_dim_mlp * 2, 128, 64] 
dropout_rate_gmf = 0.2
dropout_rate_mlp = 0.2

# Build and compile the NeuMF model
model = build_neumf_model(
    num_users=n_users,
    num_items=n_items,
    latent_dim_gmf=embedding_dim_gmf,
    latent_dim_mlp=embedding_dim_mlp,
    mlp_layers=mlp_layers,
    dropout_rate_gmf=dropout_rate_gmf,
    dropout_rate_mlp=dropout_rate_mlp
)

In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
import numpy as np

# Preparing training data
user_indices = np.array(df['user_id'])
item_indices = np.array(df['app_id'])
y_train = np.array(df['hours'])  
y_train_binary = (y_train > 0.5).astype(int) 

In [9]:
# Train the model
history = model.fit(
    [user_indices, item_indices], 
    y_train_binary,
    epochs=10, 
    batch_size=64, 
    validation_split=0.1
)

Epoch 1/10
226/226 [==============================] - 2s 6ms/step - loss: 0.2374 - accuracy: 0.9476 - val_loss: 0.0541 - val_accuracy: 0.9950
Epoch 2/10
226/226 [==============================] - 1s 3ms/step - loss: 0.1299 - accuracy: 0.9491 - val_loss: 0.0565 - val_accuracy: 0.9950
Epoch 3/10
226/226 [==============================] - 1s 3ms/step - loss: 0.1009 - accuracy: 0.9579 - val_loss: 0.0771 - val_accuracy: 0.9881
Epoch 4/10
226/226 [==============================] - 1s 3ms/step - loss: 0.0876 - accuracy: 0.9638 - val_loss: 0.0952 - val_accuracy: 0.9756
Epoch 5/10
226/226 [==============================] - 1s 3ms/step - loss: 0.0769 - accuracy: 0.9672 - val_loss: 0.0967 - val_accuracy: 0.9769
Epoch 6/10
226/226 [==============================] - 1s 3ms/step - loss: 0.0660 - accuracy: 0.9726 - val_loss: 0.1069 - val_accuracy: 0.9731
Epoch 7/10
226/226 [==============================] - 1s 2ms/step - loss: 0.0527 - accuracy: 0.9791 - val_loss: 0.1166 - val_accuracy: 0.9688
Epoch 

In [10]:
# Evaluate the model
loss, accuracy = model.evaluate([user_indices, item_indices], y_train_binary)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

501/501 [==============================] - 1s 1ms/step - loss: 0.0308 - accuracy: 0.9909
Loss: 0.030765559524297714
Accuracy: 0.990939199924469


In [11]:
def recommend_items_for_user(model, user_id, all_item_ids, df, num_recommendations=10):
    # Get all item_ids user hasn't interacted with yet
    interacted_items = df[df['user_id'] == user_id]['app_id'].tolist()
    items_to_predict = [item for item in all_item_ids if item not in interacted_items]
    
    user_array = np.array([user_id] * len(items_to_predict))
    item_array = np.array(items_to_predict)
    
    # Predict the scores for this user and the remaining items
    predictions = model.predict([user_array, item_array])
    
    # Sort the predictions to get the top items
    top_indices = predictions.flatten().argsort()[-num_recommendations:][::-1]
    top_item_ids = item_array[top_indices]
    
    return top_item_ids

all_item_ids = df['app_id'].unique()

# Example: Recommend items for a specific user
user_id = 4  # USER INPUT: Replace with the user_id you want recommendations for
top_recommendations = recommend_items_for_user(model, user_id, all_item_ids, df, num_recommendations=10)

print(f"Top recommended items for user {user_id}: {top_recommendations}")



117/117 [==============================] - 0s 1ms/step
Top recommended items for user 4: [  77 2711  159 1215 1394   47  307 1332  121   65]
